In [1]:
import sys
sys.path.append("../code")
from dataset import build_dataset
from config import DataArguments, TrainerArguments, ModelArguments

C:\Users\pha\anaconda3\envs\bnn\lib\site-packages\torch\cuda\__init__.py:80: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 10010). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at  ..\c10\cuda\CUDAFunctions.cpp:112.)
  return torch._C._cuda_getDeviceCount() > 0


In [2]:
data_args = DataArguments
training_args = TrainerArguments
model_args = ModelArguments

data_args.data_dir = "../data/"
data_args.seed = training_args.seed
data_args.max_seq_len = model_args.max_seq_len

dataset = build_dataset

[11/20/2021 18:56:07] INFO - dataloader: Initialize Train Dataset.
[11/20/2021 18:56:07] INFO - dataloader: Remove abstract.
[11/20/2021 18:56:07] INFO - dataloader: Remove task_id
[11/20/2021 18:56:07] INFO - dataloader: Use 0.05% of the total dataset.
[11/20/2021 18:56:08] INFO - dataloader: Total 2363 of papers will be used.
[11/20/2021 18:56:10] INFO - preprocess: Successfully loaded Spacy Tokenizer, en_core_web_trf
[11/20/2021 18:56:10] INFO - dataloader: Successfully loaded mapper file ..\assets\area2idx.json
[11/20/2021 18:56:10] INFO - dataloader: Train dataset was successfully loaded.


In [3]:
import torch
tokenize = dataset.preprocessor.tokenize
def encode(example):
    
    input_raw = example["title"] if "abstract" not in example else f"{example['title']}[SEP]{example['abstract']}"
    input_ids = tokenize(input_raw).squeeze()
    label = torch.tensor(dataset.area2idx[example["area"]], dtype=torch.long)

    seq_len = len(input_ids)
    if seq_len > data_args.max_seq_len:
        input_ids = input_ids[:data_args.max_seq_len]
        seq_len = data_args.max_seq_len
    else:
        zero_seq = torch.zeros(data_args.max_seq_len)
        zero_seq[:seq_len] = input_ids
        input_ids, zero_seq = zero_seq, input_ids

    return {
        "input_ids": input_ids,
        "label": label,
        "seq_len": torch.tensor(seq_len, dtype=torch.long)
    }

def batch_encode(example):
    
    input_raw = example["title"] if "abstract" not in example else f"{example['title']}[SEP]{example['abstract']}"
    input_ids = tokenize(input_raw)
    label = [dataset.area2idx[area] for area in example["area"]]

    return {
        "input_ids": input_ids,
        "label": label,
        "seq_len": [len(i) for i in input_ids]
    }

In [4]:
hf_dataset = dataset.dataset.map(batch_encode, batched=True, remove_columns=["area", "title"])

[11/20/2021 18:56:13] WARNING - datasets.fingerprint: Parameter 'function'=<function batch_encode at 0x00000211772D5828> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.
  0%|          | 0/3 [00:00<?, ?ba/s]C:\Users\pha\anaconda3\envs\bnn\lib\site-packages\torch\autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
100%|██████████| 3/3 [01:29<00:00, 29.85s/ba]


In [6]:
hf_dataset

Dataset({
    features: ['input_ids', 'label', 'seq_len'],
    num_rows: 2363
})

In [7]:
def batch_encode(example):
    
    input_raw = example["title"] if "abstract" not in example else f"{example['title']}[SEP]{example['abstract']}"
    input_ids = tokenize(input_raw)
    label = [dataset.area2idx[area] for area in example["area"]]

    return {
        "input_ids": input_ids,
        "label": label,
        # "seq_len": [len(i) for i in input_ids]
    }
    
hf_dataset = dataset.dataset.map(batch_encode, batched=True, remove_columns=["area", "title"])

  0%|          | 0/3 [00:00<?, ?ba/s]C:\Users\pha\anaconda3\envs\bnn\lib\site-packages\torch\autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
100%|██████████| 3/3 [01:31<00:00, 30.40s/ba]


In [9]:
dataset.tokenize(dataset.dataset[10]["title"]).tolist()

C:\Users\pha\anaconda3\envs\bnn\lib\site-packages\torch\autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


[[0,
  45681,
  5206,
  45998,
  3870,
  10845,
  11,
  11202,
  26970,
  21888,
  8250,
  10244,
  19,
  26056,
  7590,
  14641,
  2]]

In [10]:
dataset[100]["input_ids"].tolist()

C:\Users\pha\anaconda3\envs\bnn\lib\site-packages\torch\autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


[0,
 35166,
 17789,
 46637,
 13807,
 13,
 19268,
 12,
 46022,
 16730,
 29517,
 154,
 8,
 2825,
 21553,
 154,
 26283,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [14]:
tokenize("dddd")

C:\Users\pha\anaconda3\envs\bnn\lib\site-packages\torch\autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


tensor([[    0, 16134, 16134,     2]])

In [12]:
tokenize(dataset.dataset[998
])

In [4]:
batch = next(iter(dataloader))

C:\Users\pha\anaconda3\envs\bnn\lib\site-packages\torch\autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


In [5]:
batch

{'input_ids': tensor([[    0,   500,  4774,  ...,     0,     0,     0],
         [    0,  9685, 16101,  ...,     0,     0,     0],
         [    0, 48730,     9,  ...,     0,     0,     0],
         ...,
         [    0, 46064,    12,  ...,     0,     0,     0],
         [    0, 48739,    12,  ...,     0,     0,     0],
         [    0, 43597, 42312,  ...,     0,     0,     0]]),
 'labels': tensor([11,  7, 15,  4, 15, 10,  7,  6, 14, 14, 12,  7,  7, 10, 15,  7, 10, 14,
         10, 10,  7,  6, 13, 11,  7, 12,  7,  4,  4,  7, 15, 15,  4,  0, 12,  3,
         12,  3, 15,  3, 13,  3, 15,  3, 10, 11,  4,  4, 10,  7,  7,  7,  2,  3,
         10,  7,  3,  7,  7, 10, 12,  3, 15, 15,  4,  7, 15,  3, 12, 15, 10, 14,
         12,  4, 14, 12, 10,  3, 14,  7, 15,  2,  8,  0, 12,  4, 10, 14,  7, 15,
         10, 10, 14,  7, 10, 12,  3, 14,  3, 15, 15,  0,  3,  7, 11, 14, 14, 12,
         14, 13, 12,  7, 10, 15, 10,  3, 10,  5, 12,  5,  3, 15,  7, 10, 14, 15,
         14,  3]),
 'seq_len': tensor([ 

In [10]:
import torch
torch.vstack((dataset[0]["input_ids"], dataset[0]["input_ids"]))

C:\Users\pha\anaconda3\envs\bnn\lib\site-packages\torch\autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


torch.Size([2, 128])

In [15]:
from transformers import BertConfig, BertForSequenceClassification

In [26]:
model_config = BertConfig(
    vocab_size=len(nlp.vocab.strings)
)
model = BertForSequenceClassification(model_config)

In [22]:
model(input_ids=dataset[0]["input_ids"].unsqueeze(0))

C:\Users\pha\anaconda3\envs\bnn\lib\site-packages\torch\autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


SequenceClassifierOutput(loss=None, logits=tensor([[-0.1435, -0.0564]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [25]:
logit = model(input_ids=dataset[0]["input_ids"].unsqueeze(0))["logits"]

In [27]:
logit

tensor([[-0.1429,  0.0209]], grad_fn=<AddmmBackward0>)

In [18]:
zero_seq[0]dd[:seq_len]

tensor([0.])

In [28]:
token_seq[0]

NameError: name 'token_seq' is not defined

In [3]:
import spacy

documents = "Spacy sucks. Nothing is being donwloaded."

tokenizer = spacy.load("en_core_web_trf")
doc = tokenizer(documents)

C:\Users\pha\anaconda3\envs\bnn\lib\site-packages\torch\cuda\__init__.py:80: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 10010). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at  ..\c10\cuda\CUDAFunctions.cpp:112.)
  return torch._C._cuda_getDeviceCount() > 0
C:\Users\pha\anaconda3\envs\bnn\lib\site-packages\torch\autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


In [16]:
type(doc._.trf_data.tokens["input_ids"])

torch.Tensor